# Profile the two gap methods

In [ ]:
from curses.textpad import rectangle

from gap_study_utils.analysis_data import AnalysisData
from gap_study_utils.gaps.gap_type import GapType
import numpy as np
import os
from gap_study_utils.analysis_data import AnalysisData, get_suggested_tmax
from gap_study_utils.utils.signal_utils import waveform
from gap_study_utils.gaps.gap_funcs import generate_gap_ranges
from gap_study_utils.gaps import GapType
from tqdm.auto import tqdm


import numpy as np





LN_A = np.log(1e-21)
LN_F = np.log(0.005)
LN_FDOT = np.log(1e-9)
TRUE = [LN_A, LN_F, LN_FDOT]

HOURS = 60 * 60
DAYS = 24 * HOURS

np.random.seed(0)
dt = 10


def collect_runtimes(ndays:int):
    tmax = get_suggested_tmax(DAYS * ndays)
    gap_ranges = generate_gap_ranges(tmax, gap_period=DAYS * 14, gap_duration=HOURS * 7)
    kwgs = dict(
        data_kwargs=dict(dt=dt, noise=False, tmax=tmax),
        gap_kwargs=dict(type=GapType.RECTANGULAR_WINDOW, gap_ranges=gap_ranges),
        waveform_generator=waveform,
        waveform_parameters=TRUE,
    );
    
    kwgs['gap_kwargs']['type'] = GapType.RECTANGULAR_WINDOW
    rectangle_time = %timeit -o AnalysisData(**kwgs)
    
    kwgs['gap_kwargs']['type'] = GapType.STITCH
    stitch_time = %timeit -o AnalysisData(**kwgs)
    return dict(
        days=ndays,
        rectangle_avg=rectangle_time.average,
        rectangle_stdev=rectangle_time.stdev,
        stitch_avg=stitch_time.average,
        stitch_stdev=stitch_time.stdev,
    )


In [ ]:
%%capture


days = [16, 32, 64, 128]
runtime_data = [collect_runtimes(day) for day in tqdm(days)]


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

data = pd.DataFrame(runtime_data)

fig, ax = plt.subplots(figsize=(4, 2.5))
ax.set_xlabel('Number of days')
ax.set_ylabel('Runtime (s)')

# Fit the curves
degree = 3  # You can change the degree of the polynomial
p_rect = np.polyfit(data['days'], data['rectangle_avg'], degree)
p_stitch = np.polyfit(data['days'], data['stitch_avg'], degree)

# Extrapolate the polynomial models
days_fit = np.linspace(data['days'].min(), 1000, 100)  # Extend the range
rect_fit = np.polyval(p_rect, days_fit)
stitch_fit = np.polyval(p_stitch, days_fit)


# Plot original data with error bars
ax.errorbar(data['days'], data['rectangle_avg'], yerr=data['rectangle_stdev'], fmt='o', color='tab:blue', label='Rectangle')
# ax.plot(days_fit, rect_fit, label=f'Rectangle (degree {degree})', color='tab:blue')

ax.errorbar(data['days'], data['stitch_avg'], yerr=data['stitch_stdev'], fmt='o', color='tab:orange')
ax.loglog(days_fit, stitch_fit, label=f'Stitch', color='tab:orange')



ax.legend()
plt.savefig('figures/profile_methods.png', dpi=300, bbox_inches='tight')

![](figures/profile_methods.png)